# Fuzzy String Matching untuk Data Transaksi

## Deskripsi
Notebook ini mendemonstrasikan penggunaan **fuzzy string matching** untuk mencocokkan data transaksi dengan profil pelanggan dan master produk. Fuzzy matching berguna ketika data tidak persis sama tetapi memiliki kemiripan (typo, variasi penulisan, dll).

## Tujuan
1. **Mencocokkan nama pelanggan** di transaksi dengan nama lengkap di profil pelanggan
2. **Mencocokkan alamat pengiriman** dengan alamat domisili untuk validasi
3. **Mencocokkan nama barang** di transaksi dengan nama produk resmi di master produk
4. **Menghasilkan tabel final** yang sudah diperkaya dengan informasi lengkap

## Library yang Digunakan
- `pandas` - untuk manipulasi data
- `thefuzz` - untuk fuzzy string matching (pengganti FuzzyWuzzy)
- `python-Levenshtein` - untuk performa fuzzy matching yang lebih cepat

## Struktur Data
Notebook ini menggunakan 3 sheet dari file Excel:
- **profil_pelanggan**: Data profil lengkap pelanggan
- **transaksi_pembelian**: Data transaksi yang akan diperkaya
- **master_produk**: Data produk resmi sebagai referensi

## Algoritma Fuzzy Matching
- **token_sort_ratio**: Untuk mencocokkan nama (mengabaikan urutan kata)
- **token_set_ratio**: Untuk mencocokkan produk dan alamat (mengabaikan duplikasi kata)

## Threshold Scoring
- **Skor Nama**: ≥ 70 (dari skala 0-100)
- **Skor Alamat**: ≥ 75 (dari skala 0-100)  
- **Skor Produk**: ≥ 75 (dari skala 0-100)

## Cara Menjalankan

### Langkah 1: Install Dependencies
Jalankan cell pertama untuk menginstall library yang diperlukan:
```python
%pip install thefuzz python-Levenshtein
```

### Langkah 2: Import dan Setup
Jalankan cell berikutnya yang berisi import library dan setup path file Excel.

### Langkah 3: Eksekusi Fuzzy Matching
Kode akan secara otomatis:
1. **Membaca data** dari 3 sheet Excel
2. **Tahap 1**: Mencocokkan transaksi dengan profil pelanggan
3. **Tahap 2**: Mencocokkan hasil dengan master produk  
4. **Menampilkan hasil** final yang sudah diperkaya

### Output yang Dihasilkan
- Analisis skor sebelum filtering
- Tabel final dengan kolom:
  - Data transaksi original
  - Data profil pelanggan yang cocok
  - Data produk resmi yang cocok
  - Skor kemiripan untuk setiap matching

---

## Kode Implementation

In [1]:
import pandas as pd
from thefuzz import process, fuzz
import os

# ===================================================================
# MEMBACA DATA DARI FILE EXCEL ANDA
# ===================================================================
file_path = r"C:\Users\user\OneDrive - untirta.ac.id\UNTIRTA\Bahan Ajar\Pengantar Data Sains\25-26\Contoh Data Penjualan Toko.xlsx"

if not os.path.exists(file_path):
    print(f"Error: File tidak ditemukan di path: {file_path}")
    exit()

df_profil = pd.read_excel(file_path, sheet_name='profil_pelanggan')
df_transaksi = pd.read_excel(file_path, sheet_name='transaksi_pembelian')
df_produk = pd.read_excel(file_path, sheet_name='master_produk')

print("--- Data Berhasil Dimuat dari Excel ---")

# ===================================================================
# LOGIKA FUZZY MATCHING
# ===================================================================

def find_best_match(name, choices_list, scorer=fuzz.token_set_ratio):
    best_match = process.extractOne(name, choices_list, scorer=scorer)
    if best_match:
        return best_match[0], best_match[1]
    return None, None

# TAHAP 1
print("\n" + "="*50 + "\n")
print("TAHAP 1: Mencocokkan Transaksi dengan Profil (Nama & Alamat)...")

choices_nama = df_profil['nama_lengkap'].tolist()
match_results_nama = df_transaksi['nama_pelanggan'].apply(lambda x: find_best_match(x, choices_nama))
df_transaksi[['profil_cocok', 'skor_nama']] = pd.DataFrame(match_results_nama.tolist(), index=df_transaksi.index)

df_transaksi_intermediate = pd.merge(df_transaksi, df_profil, left_on='profil_cocok', right_on='nama_lengkap', how='left')

def calculate_address_score(row):
    if pd.notna(row['alamat_pengiriman']) and pd.notna(row['alamat_domisili']):
        return fuzz.token_set_ratio(row['alamat_pengiriman'], row['alamat_domisili'])
    return 0
df_transaksi_intermediate['skor_alamat'] = df_transaksi_intermediate.apply(calculate_address_score, axis=1)


print("\n--- Analisis Skor (Sebelum Filter) ---")
print(df_transaksi_intermediate[['id_transaksi', 'nama_pelanggan', 'profil_cocok', 'skor_nama', 'skor_alamat']].to_string())


# Filter berdasarkan skor nama DAN skor alamat
# =================================================================
SKOR_NAMA_THRESHOLD = 70 
# =================================================================
SKOR_ALAMAT_THRESHOLD = 75

df_linked_profil = df_transaksi_intermediate[
    (df_transaksi_intermediate['skor_nama'] >= SKOR_NAMA_THRESHOLD) &
    (df_transaksi_intermediate['skor_alamat'] >= SKOR_ALAMAT_THRESHOLD)
].copy() 


# TAHAP 2
print("\nTAHAP 2: Mencocokkan Transaksi dengan Daftar Produk...")

choices_produk = df_produk['nama_produk_resmi'].tolist()
match_results_produk = df_linked_profil['nama_barang'].apply(lambda x: find_best_match(x, choices_produk, scorer=fuzz.token_set_ratio))
df_linked_profil.loc[:, 'produk_cocok'] = match_results_produk.apply(lambda x: x[0] if x else None)
df_linked_profil.loc[:, 'skor_produk'] = match_results_produk.apply(lambda x: x[1] if x else 0)

df_final = pd.merge(df_linked_profil, df_produk, left_on='produk_cocok', right_on='nama_produk_resmi', how='left')

SKOR_PRODUK_THRESHOLD = 75
df_final = df_final[df_final['skor_produk'] >= SKOR_PRODUK_THRESHOLD]

# TAHAP AKHIR
print("\n" + "="*50 + "\n")
print("HASIL AKHIR: Tabel Transaksi yang Diperkaya dan Rapi (HASIL FINAL)")
final_columns = [
    'id_transaksi', 'id_profil', 'nama_pelanggan', 'nama_lengkap',
    'id_produk', 'nama_barang', 'nama_produk_resmi',
    'jumlah', 'harga', 'skor_nama', 'skor_alamat', 'skor_produk'
]
final_columns_exist = [col for col in final_columns if col in df_final.columns]
print(df_final[final_columns_exist].to_string())

--- Data Berhasil Dimuat dari Excel ---


TAHAP 1: Mencocokkan Transaksi dengan Profil (Nama & Alamat)...

--- Analisis Skor (Sebelum Filter) ---
  id_transaksi   nama_pelanggan            profil_cocok  skor_nama  skor_alamat
0        TRX01       Ferdian B.  Ferdian Bangkit Wijaya         88           91
1        TRX02         Weksi B.           Weksi Budiaji         83           94
2        TRX03  Chintia Lestari    Cynthia Dewi Lestari         74           93
3        TRX04    Ahmad Zaelany           Ahmad Zaelani         92           81
4        TRX05    Weksi Budiaji           Weksi Budiaji        100           77

TAHAP 2: Mencocokkan Transaksi dengan Daftar Produk...


HASIL AKHIR: Tabel Transaksi yang Diperkaya dan Rapi (HASIL FINAL)
  id_transaksi id_profil   nama_pelanggan            nama_lengkap id_produk             nama_barang        nama_produk_resmi  jumlah  harga  skor_nama  skor_alamat  skor_produk
0        TRX01      P001       Ferdian B.  Ferdian Bangkit Wijaya    PRO

In [2]:
# ===================================================================
# MENAMPILKAN HASIL DALAM FORMAT DATAFRAME
# ===================================================================

# Cek apakah variabel yang diperlukan sudah ada
if 'df_final' not in locals():
    print("⚠️  ERROR: Variabel belum tersedia!")
    print("🔧 SOLUSI: Jalankan cell sebelumnya terlebih dahulu (cell 3) yang berisi:")
    print("   • Import library")
    print("   • Membaca data Excel") 
    print("   • Proses fuzzy matching")
    print("   • Pembuatan variabel df_final")
    print("\n" + "="*70)
    print("LANGKAH YANG BENAR:")
    print("1️⃣  Jalankan Cell 3 (kode fuzzy matching)")
    print("2️⃣  Kemudian jalankan Cell 4 ini (tampilan hasil)")
    print("="*70)
else:
    print("\n" + "="*70)
    print("RINGKASAN HASIL FUZZY MATCHING")
    print("="*70)

    # Membuat DataFrame ringkasan hasil
    if not df_final.empty:
        # Kolom yang akan ditampilkan dalam hasil akhir
        display_columns = [
            'id_transaksi', 'id_profil', 'nama_pelanggan', 'nama_lengkap',
            'id_produk', 'nama_barang', 'nama_produk_resmi',
            'jumlah', 'harga', 'skor_nama', 'skor_alamat', 'skor_produk'
        ]
        
        # Filter kolom yang benar-benar ada di DataFrame
        available_columns = [col for col in display_columns if col in df_final.columns]
        
        # Membuat DataFrame hasil akhir
        result_df = df_final[available_columns].copy()
        
        # Menampilkan informasi statistik
        print(f"\n📊 STATISTIK HASIL:")
        print(f"   • Total transaksi yang berhasil dicocokkan: {len(result_df)}")
        print(f"   • Rata-rata skor nama: {result_df['skor_nama'].mean():.1f}")
        print(f"   • Rata-rata skor alamat: {result_df['skor_alamat'].mean():.1f}")
        print(f"   • Rata-rata skor produk: {result_df['skor_produk'].mean():.1f}")
        
        print(f"\n📋 HASIL AKHIR FUZZY MATCHING:")
        print("-" * 70)
        
        # Menampilkan DataFrame dengan format yang rapi
        pd.set_option('display.max_columns', None)
        pd.set_option('display.width', None)
        pd.set_option('display.max_colwidth', 30)
        
        # Menampilkan DataFrame
        display(result_df)
        
    else:
        print("\n⚠️  TIDAK ADA DATA YANG COCOK")
        print("   Tidak ada transaksi yang memenuhi threshold yang ditetapkan.")
        print("   Pertimbangkan untuk menurunkan nilai threshold:")
        print(f"   • Skor nama: {SKOR_NAMA_THRESHOLD}")
        print(f"   • Skor alamat: {SKOR_ALAMAT_THRESHOLD}")
        print(f"   • Skor produk: {SKOR_PRODUK_THRESHOLD}")

    print("\n" + "="*70)
    print("FUZZY MATCHING SELESAI")
    print("="*70)


RINGKASAN HASIL FUZZY MATCHING

📊 STATISTIK HASIL:
   • Total transaksi yang berhasil dicocokkan: 5
   • Rata-rata skor nama: 87.4
   • Rata-rata skor alamat: 87.2
   • Rata-rata skor produk: 89.2

📋 HASIL AKHIR FUZZY MATCHING:
----------------------------------------------------------------------


,id_transaksi,id_profil,nama_pelanggan,nama_lengkap,id_produk,nama_barang,nama_produk_resmi,jumlah,harga,skor_nama,skor_alamat,skor_produk
0,TRX01,P001,Ferdian B.,Ferdian Bangkit Wijaya,PROD-A,Kopi Gula Aren 1 Liter,Kopi Susu Gula Aren 1L,1,55000,88,91,78
1,TRX02,P004,Weksi B.,Weksi Budiaji,PROD-B,Roti Sobek Coklat,Roti Sobek Cokelat Keju,2,25000,83,94,85
2,TRX03,P002,Chintia Lestari,Cynthia Dewi Lestari,PROD-C,Donat Klasik 6pcs,Donat Gula Klasik Isi 6,1,45000,74,93,83
3,TRX04,P003,Ahmad Zaelany,Ahmad Zaelani,PROD-A,Kopi Susu Aren 1L,Kopi Susu Gula Aren 1L,1,55000,92,81,100
4,TRX05,P004,Weksi Budiaji,Weksi Budiaji,PROD-D,Teh Melati 500ml,Teh Melati Segar 500ml,3,15000,100,77,100



FUZZY MATCHING SELESAI
